In [1]:
import ktrain
from ktrain import text as txt

## Reading and preprocessing input

In [2]:
train_docs = []
train_labels = []
for item in open("../data/datasets_bert/firststage_se-nse/train.tsv", 'r'):
    item_fields = item.strip().split('\t')
    train_docs.append(item_fields[0].strip())
    train_labels.append(item_fields[1].strip())
    
test_docs = []
test_labels = []
for item in open("../data/datasets_bert/firststage_se-nse/test.tsv", 'r'):
    item_fields = item.strip().split('\t')
    test_docs.append(item_fields[0].strip())
    test_labels.append(item_fields[1].strip())

In [ ]:
# using last couple of conversations as validation set

valid_docs = train_docs[1502:]
valid_labels = train_labels[1502:]

train_docs = train_docs[:1502]
train_labels = train_labels[:1502]

print(len(train_docs))
print(len(train_labels))

print(train_docs[0])
print(train_labels[0])

print(len(valid_docs))
print(len(valid_labels))

print(valid_docs[0])
print(valid_labels[0])

In [ ]:
MODEL_NAME = 'nboost/pt-bert-base-uncased-msmarco'
t = txt.Transformer(MODEL_NAME, maxlen=500, class_names=['pos', 'neg'])
trn = t.preprocess_train(train_docs, train_labels)
val = t.preprocess_test(valid_docs, valid_labels)
test = t.preprocess_test(test_docs, test_labels)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

# Find a good learning rate

In [ ]:
# find good learning rate
learner.lr_find()             # briefly simulate training to find good learning rate
learner.lr_plot()             # visually identify best learning rate

# Train the classifier

In [ ]:
# train using 1-cycle learning rate schedule for N epochs
learner.fit_onecycle(lr=4e-5, epochs=100, checkpoint_folder="pt-bert-base-uncased-msmarco-weights_01")

# Reload weights and predict (confusion matrix)

In [5]:
# model can be downloaded here: http://hpc.isti.cnr.it/~nardini/adaptive-utterance/first_se-vs-nse.hdf5
# $ wget http://hpc.isti.cnr.it/~nardini/adaptive-utterance/first_se-vs-nse.hdf5

model.load_weights('first_se-vs-nse.hdf5')
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)
learner.validate(val_data=test, class_names=["NSE", "SE"])

              precision    recall  f1-score   support

         NSE       0.94      0.92      0.93       126
          SE       0.86      0.88      0.87        68

    accuracy                           0.91       194
   macro avg       0.90      0.90      0.90       194
weighted avg       0.91      0.91      0.91       194



array([[116,  10],
       [  8,  60]])